dask_gateway is required to setup the connection to the cluster. Otherwise, xarray has builin support for dask so no further magic is required.dask_gateway is required to setup the connection to the cluster. Otherwise, xarray has builin support for dask so no further magic is required.

In [ ]:
%matplotlib inline

import os
import pathlib

import dask_gateway

import dask.array as da
import numpy as np
import time

Connect to the the dask_gateway VM. Authentiaction is via a token available from the notebook's environment. If dask-gatway is used from the command line, it is neccessary to supply a token here.Connect to the the dask_gateway VM. Authentiaction is via a token available from the notebook's environment.

In [ ]:
gw = dask_gateway.Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")

Have the gateway create a new dask cluster. This creates a dask scheduler job in LOTUS running as the user running the notebook. Since LOTUS jobs have to queue, we re-use our cluster rather than destroying it at the end of the script.Have the gateway create a new dask cluster. This creates a dask scheduler job in LOTUS running as the user running the notebook. Since LOTUS jobs have to queue, we re-use our cluster rather than destroying it at the end of the script.

In [ ]:
clusters = gw.list_clusters()
if not clusters:
    cluster = gw.new_cluster(shutdown_on_close=False)
else:
    cluster = gw.connect(clusters[0].name)

Scale the cluster to have at least one and at most three workers. Each of these workers becomes it's own job in LOTUS, and is managed my the scheduler created in the last step. These workers have 4 cores and 16GiB of RAM each, but it is possible to ask for less.

In [ ]:
cluster.adapt(minimum=1, maximum=3)

Now that we have a stood-up cluster, we can get a client with which to inteact with it.Now that we have a stood-up cluster, we can get a client with which to inteact with it.

In [ ]:
client = cluster.get_client()

Inspecting the client object will give the dashboard URL, which is proxied from the scheduler job via the gateway VM.


In [ ]:
client

In [ ]:
# from https://j23414.github.io/jekyll_rtd/geospatial/session3-intro-to-python-dask-on-ceres.html
def calc_pi_mc(size_in_bytes, chunksize_in_bytes=200e6):  
    """Calculate PI using a Monte Carlo estimate."""
    
    size = int(size_in_bytes / 8)
    chunksize = int(chunksize_in_bytes / 8)
    
    xy = da.random.uniform(0, 1, size=(size / 2, 2), chunks=(chunksize / 2, 2))     
    
    in_circle = ((xy ** 2).sum(axis=-1) < 1)
    pi = 4 * in_circle.mean()
    return pi

def print_pi_stats(size, pi, time_delta):  
    """Print pi, calculate offset from true value, and print some stats."""
    print(f"{size / 1e9} GB\n"
          f"\tMC pi: {pi : 13.11f}"
          f"\tErr: {abs(pi - np.pi) : 10.3e}\n"
          f"\t\tTime: {time_delta : 7.3f}s")


Computations happen in dask due to the use of a dask array in the above definition. Subsequent operations on the dask array object automagically happen using dask.
While the following is running, go explore the daskboard linked above!

In [ ]:
for size in (1e9 * n for n in (1, 10, 100, 1000)):
    start = time.time()
    pi = calc_pi_mc(size).compute()
    elaps = time.time() - start
    print_pi_stats(size, pi, time_delta=elaps)